In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import scipy
import scipy.io
import scipy.sparse as sp
import matplotlib.pyplot as plt
import time
import datetime
%load_ext autoreload
%autoreload 2

In [2]:
# import scikit-surprise stuff
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import CoClustering
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import Reader
from surprise import Dataset
from surprise import similarities
from surprise.model_selection import cross_validate, GridSearchCV
from surprise.model_selection import train_test_split
from surprise import accuracy

In [3]:
# import pandas for DataFrame
import pandas as pd

In [4]:
# import costum stuff
from utility import *

In [5]:
# prepare train set file in correct format for scikit-surprise
data_df = loadData2df

In [6]:
datafilepath = "../data/data_train.csv"
data_df = loadData2df(datafilepath)

In [7]:
data_ds = loadData2ds(data_df)

In [8]:
predinfilepath = "../data/sample_submission.csv"
pred_df = loadData2df(predinfilepath)

In [9]:
full_train_ds = data_ds.build_full_trainset()

**ZSCORE**

In [10]:
algo = KNNWithZScore(k=1000, min_k=10, sim_options={'name': 'pearson', 'user_based': False}, verbose=True)

In [11]:
algo.fit(full_train_ds)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [12]:
pred = loadData2df("../data/sample_submission.csv")

In [13]:
generatePredictions(algo,pred)

In [15]:
exportPredictions(pred,outfilepath="../data/submissions/knnwithzscore.csv",add_date=True,add_time=True)

**Means**

In [16]:
algo = KNNWithMeans(k=200, min_k=10, sim_options={'name': 'pearson', 'user_based': False}, verbose=True)

In [17]:
algo.fit(full_train_ds)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [18]:
pred = loadData2df("../data/sample_submission.csv")

In [19]:
generatePredictions(algo,pred)

In [21]:
exportPredictions(pred,outfilepath="../data/submissions/knnwithmeans.csv",add_date=True,add_time=True)

### Jules super duper parameters

In [43]:
gridSVD_ne2000 = {\
'n_factors': [125],\
'lr_all': [0.01],\
'reg_all': [0.1],\
'n_epochs': [2000],\
'biased': [True]}

In [44]:
gridsearch = GridSearchCV(algo_class=SVD,param_grid=gridSVD_ne2000,measures=['RMSE'], cv=5, n_jobs=-1,joblib_verbose=2)

In [45]:
gridsearch.fit(data_ds)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 

In [ ]:
print(gridsearch.best_score['rmse'])
print(gridsearch.best_params['rmse'])
results_cv = pd.DataFrame.from_dict(gridsearch.cv_results)
results_cv_path = "../data/cvresults/cv-SVD-nepochs2000-2.csv"
results_cv_path = addDateAndTime(results_cv_path)
results_cv.to_csv(path_or_buf=results_cv_path,mode="w")
print(results_cv_path)

In [28]:
print(gridsearch.best_score['rmse'])
print(gridsearch.best_params['rmse'])

0.9938723893523729
{'n_factors': 10, 'lr_all': 0.01, 'reg_all': 0.1, 'n_epochs': 2000, 'biased': True}


In [29]:
results_cv = pd.DataFrame.from_dict(gridsearch.cv_results)

In [30]:
results_cv_path = "../data/cvresults/cv-SVD-nepochs2000.csv"
results_cv_path = addDateAndTime(results_cv_path)
results_cv.to_csv(path_or_buf=results_cv_path,mode="w")

In [31]:
print(results_cv_path)

../data/cvresults/cv-SVD-nepochs2000_2018-12-20_0357.csv


In [37]:
algo = gridsearch.best_estimator["rmse"]

In [38]:
algo.fit(full_train_ds)

In [39]:
pred = loadData2df("../data/sample_submission.csv")

In [40]:
generatePredictions(algo,pred)

In [41]:
exportPredictions(pred,outfilepath="../data/submissions/SVD-nepchos2000.csv",add_date=True,add_time=True)